In [23]:

import pandas as pd
from dateutil.relativedelta import *
from TimeBasedCV import TimeBasedCV


In [24]:
transaction=pd.read_parquet('data\\HM_parquet\\transactions_train.parquet')
tscv = TimeBasedCV(train_period=30,
                   test_period=7,
                   freq='days')
    

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_9312/297288424.py, line 5)

In [13]:
tscv.get_n_splits()

24

In [ ]:
# 機器學習範例
from sklearn.linear_model import LinearRegression
import numpy as np

X = transaction[['record_date',columns]]
y = transaction[label]

scores = []
for train_index, test_index in tscv.split(X, date_column='t_dat', stride=30):

    data_train   = X.loc[train_index].drop('record_date', axis=1)
    target_train = y.loc[train_index]

    data_test    = X.loc[test_index].drop('record_date', axis=1)
    target_test  = y.loc[test_index]

    # if needed, do preprocessing here

    clf = LinearRegression()
    clf.fit(data_train,target_train)

    preds = clf.predict(data_test)

    # accuracy for the current fold only    
    r2score = clf.score(data_test,target_test)

    scores.append(r2score)

# this is the average accuracy over all folds
average_r2score = np.mean(scores)

# implicit.ALS

In [26]:
import os; os.environ['OPENBLAS_NUM_THREADS']='1'
import numpy as np
import pandas as pd
import implicit
from scipy.sparse import coo_matrix
from implicit.evaluation import mean_average_precision_at_k


In [27]:
dfu = pd.read_parquet('data/HM_parquet/customers.parquet')
dfi = pd.read_parquet('data/HM_parquet/articles.parquet')
ALL_USERS = dfu['customer_id'].unique().tolist()
ALL_ITEMS = dfi['article_id'].unique().tolist()

user_ids = dict(list(enumerate(ALL_USERS)))
item_ids = dict(list(enumerate(ALL_ITEMS)))

user_map = {u: uidx for uidx, u in user_ids.items()}
item_map = {i: iidx for iidx, i in item_ids.items()}
# 將落落長的使用者id和商品id轉為編號
transaction['user_id'] = transaction['customer_id'].map(user_map)
transaction['item_id'] = transaction['article_id'].map(item_map)

del dfu, dfi

In [29]:
def to_user_item_coo(df, user, item):
    """
    Turn a dataframe with transactions into a COO sparse items x users matrix

    Args:
        df ([pandas.DataFrame])
        user ([pandas.Series])
        item ([pandas.Series])

    Returns:
        coo ([]): 
    """    
    row = user.values
    col = item.values
    data = np.ones(df.shape[0])
    coo = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
    return coo

def get_val_matrices(data_train, data_val):
    """ Split into training and validation and create various matrices
        
        Returns a dictionary with the following keys:
            coo_train: training data in COO sparse format and as (users x items)
            csr_train: training data in CSR sparse format and as (users x items)
            csr_val:  validation data in CSR sparse format and as (users x items)
    
    """
    coo_train = to_user_item_coo(data_train)
    coo_val = to_user_item_coo(data_val)

    csr_train = coo_train.tocsr()
    csr_val = coo_val.tocsr()
    
    return {'coo_train': coo_train,
            'csr_train': csr_train,
            'csr_val': csr_val
          }


def validate(matrices, factors=200, iterations=20, regularization=0.01, show_progress=True):
    """ Train an ALS model with <<factors>> (embeddings dimension) 
    for <<iterations>> over matrices and validate with MAP@12
    """
    coo_train, csr_train, csr_val = matrices['coo_train'], matrices['csr_train'], matrices['csr_val']
    
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 iterations=iterations, 
                                                 regularization=regularization, 
                                                 random_state=42)
    model.fit(coo_train, show_progress=show_progress)
    
    # The MAPK by implicit doesn't allow to calculate allowing repeated items, which is the case.
    # TODO: change MAP@12 to a library that allows repeated items in prediction
    map12 = mean_average_precision_at_k(model, csr_train, csr_val, K=12, show_progress=show_progress, num_threads=4)
    # print(f"Factors: {factors:>3} - Iterations: {iterations:>2} - Regularization: {regularization:4.3f} ==> MAP@12: {map12:6.5f}")
    return map12

In [31]:
def train_ALS(data_train,data_val):
    matrices = get_val_matrices(data_train=data_train, data_val=data_val)
    best_map12 = 0
    best_params = ""
    for factors in [40, 50, 60, 100, 200, 500, 1000]:
        for iterations in [3, 12, 14, 15, 20]:
            for regularization in [0.01]:
                map12 = validate(matrices, factors, iterations, regularization, show_progress=False)
                if map12 > best_map12:
                    best_map12 = map12
                    best_params = f"Best MAP@12 = {best_map12}. factors: {factors}, iterations: {iterations}, regularization: {regularization}"
    print(best_params)

In [ ]:
best_map12 = 0
best_params = ""
    for factors in [40, 50, 60, 100, 200, 500, 1000]:
        for iterations in [3, 12, 14, 15, 20]:
            for regularization in [0.01]:
                scores = []
                for train_index, val_index in tscv.split(transaction, date_column='t_dat', stride=30):
                    
                    data_train = transaction.loc[train_index]
                    data_val = transaction.loc[val_index]
                    matrices = get_val_matrices(data_train, data_val)
                    
                    

                    scores.append(r2score)

                # this is the average accuracy over all folds
                average_r2score = np.mean(scores)

In [ ]:

# How to use TimeBasedCV
data_for_modeling=pd.read_csv('data.csv', parse_dates=['record_date'])
tscv = TimeBasedCV(train_period=30,
                   test_period=7,
                   freq='days')
for train_index, test_index in tscv.split(data_for_modeling,
                   validation_split_date=datetime.date(2019,2,1), date_column='record_date'):
    print(train_index, test_index)

# get number of splits
tscv.get_n_splits()

#### Example- compute average test sets score: ####
X = data_for_modeling[['record_date',columns]]
y = data_for_modeling[label]
from sklearn.linear_model import LinearRegression
import numpy as np

scores = []
for train_index, test_index in tscv.split(X, validation_split_date=datetime.date(2019,2,1)):

    data_train   = X.loc[train_index].drop('record_date', axis=1)
    target_train = y.loc[train_index]

    data_test    = X.loc[test_index].drop('record_date', axis=1)
    target_test  = y.loc[test_index]

    # if needed, do preprocessing here

    clf = LinearRegression()
    clf.fit(data_train,target_train)

    preds = clf.predict(data_test)

    # accuracy for the current fold only    
    r2score = clf.score(data_test,target_test)

    scores.append(r2score)

# this is the average accuracy over all folds
average_r2score = np.mean(scores)
#### End of example ####

#### Example- RandomizedSearchCV ####
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMRegressor
from random import randint, uniform

tscv = TimeBasedCV(train_period=10, test_period=3)
index_output = tscv.split(data_for_modeling, validation_split_date=datetime.date(2019,2,1))

lgbm = LGBMRegressor()

lgbmPd = {" max_depth": [-1,2]
         }

model = RandomizedSearchCV(
    estimator = lgbm,
    param_distributions = lgbmPd,
    n_iter = 10,
    n_jobs = -1,
    iid = True,
    cv = index_output,
    verbose=5,
    pre_dispatch='2*n_jobs',
    random_state = None,
    return_train_score = True)

model.fit(X.drop('record_date', axis=1),y)
model.cv_results_
#### End of example ####
